In [80]:
from mesa import Agent, Model
from mesa.time import StagedActivation
import random
import numpy as np

random.seed(42)

In [81]:
## Parameters 
N = 10
prop_adversaries = .2
coin_toss = .8
rounds = 20

In [82]:
class Model(Model):
    def __init__(self, N, coin_toss = coin_toss):
        
        super().__init__()

        assert N > 0, "N must be greater than 0."
        self.num_agents = N

        ## Staged activation to determine which round
        ## And whose turn to be the chooser
        self.schedule = StagedActivation(self, stage_list=["choose", "vote", "act", "update"])
        self.round = 0 
        self.whose_turn = self.round % N

        ## if anyone was chosen
        self.chose = False
        ## who is the chosen, and -1 flags is that no one was
        self.chosen = []

        ## voting record of across the rounds
        self.voting_record = []
        ## voting record of the current round
        self.votes_round = []

        ## Action
        self.action_record = []
        self.action_round = [-1,-1]

        ## Outcome 
        self.outcome = []

        ## Coin Toss
        self.coin_toss = coin_toss
        
        # Change this up a little
        for i in range(self.num_agents):  # Example: creating 10 agents
            agent = DummyAgent(i, self)
            self.schedule.add(agent)

        

    ## CONVENION :: check_* are methods for agents to check certain aspects of the model
    
    """
    CHECKING TURNS and CHOOSER-CHOSEN

    These methods concern the first stage of the model
    where the agents choose or are chosen
    """
    def check_whose_turn(self):
        return self.whose_turn
    
    def check_who_was_chosen(self):
        return self.chosen[self.round]
    
    # check all choose chosen pairs
    def check_historical_choose_chosen_pairs(self):
        
        chooser = [ i % self.num_agents for i in list(range(0, self.round)) ]
        chosen = self.chosen

        choose_chosen_pair = list(zip(chooser, chosen))

        return choose_chosen_pair
    
    """
    CHECKING VOTING

    These methods concern information of voting outcomes
    historical and current
    """
    def check_historical_voting_records(self):
        return self.voting_record
    
    def check_current_voting_record(self):
        return self.votes_round
    
    def check_agent_voting_record(self, agent):
        return [ i[agent] for i in self.voting_record ]
    
    def check_vote_pass(self):

        votes = self.votes_round

        if sum(votes)/len(votes) > 0.5:
            return True
        else:
            return False
    
    """
    CHECKING OUTCOME

    These methods concern information of outcomes
    historical and current
    """
    def check_outcome_current_round(self):
        return self.outcome[-1]
    
    def check_outcome(self):
        return self.outcome
        
    def toss_coin(self):
        # Using random.choices() for a single selection
        toss = random.choices([0,1], weights=[self.coin_toss, 1 - self.coin_toss], k=1)[0]
        return toss
    
    """
    DETERMINE OUTCOME

    Determine Outcomes from Actions
    """
    def outcome_determination(self):
        ## determine which team won in given round
        ## 1 = Good team wins, 0 = Bad team wins
        if self.action_round[0] == 1 and self.action_round[1] == 1:
            self.outcome.append(1)
        elif self.action_round[0] == -1 and self.action_round[1] == -1:
            self.outcome.append(self.toss_coin())
        else:
            self.outcome.append(0)

   
    def step(self):
        self.schedule.step()
        self.round += 1
        
        ## Reset Parameters
        self.whose_turn = self.round % N
        self.chose = False
        self.voting_record.append(self.votes_round)
        self.votes_round = []
        self.action_record.append(self.action_round)
        self.action_round = [-1,-1]



In [83]:
class DummyAgent(Agent):
    def __init__(self, id, model):

        ## Apparently this is always needed
        super().__init__(id, model)

        ## Every agent has access to the model, bad design, I know, but I am lazy :( 
        self.model_state = model
        ## Unique ID, based on integer
        self.id = id

        ## This is the list of "others" ID to choose from
        others = list(range(0,N))
        others.remove(self.id)
        ## It is the belief that an agent is GOOD
        beliefs = [1] *  (N - 1)
        
        self.belief = dict(zip(others, beliefs))

    ## DO NOT CHANGE
    def select_from_belief(self):

        ## For conversion to a valid probability distribution
        normalizing_constant = sum(list(self.belief.values()))
        
        ## If the agent does not trust anyone, then
        if normalizing_constant == 0:
            ## No one was chosen
            return -1
        
        else:

            ## Tell model that someone is going to be chosen
            self.model_state.chose = True
            selection_probability = [b / normalizing_constant for b in self.belief.values()]
            # Using random.choices() for a single selection
            selected_item = random.choices(list(self.belief.keys()), weights=selection_probability, k=1)[0]

            return selected_item
    
    def choose(self):

        # Implementation of choose action
        if self.model_state.check_whose_turn() == self.id:

            choice = self.select_from_belief()
            self.model_state.chosen.append(choice)
            
        else:
            pass

    
    def vote(self):
        
        ## if someone is chosen
        if self.model_state.check_who_was_chosen() != -1:

            chooser = self.model_state.check_whose_turn()
            chosen = self.model_state.check_who_was_chosen()

            belief_chooser, belief_chosen = None, None

            ## If you are the chooser
            if chooser == self.id:
                belief_chooser = 1
            ## If you are the chosen
            elif chosen == self.id:
                belief_chosen = 1
            
            ## What to do if you are neither
            ## Then look at my current belief
            if belief_chooser is None:
                belief_chooser = self.belief[chooser]
            if belief_chosen is None:
                belief_chosen = self.belief[chosen]

            ## What is your vote?
            if belief_chooser + belief_chosen > 1:
                ## TODO
                current_vote = 1
            else:
                ## TODO
                current_vote = 0

        ## if nobody was chosen
        else: 
            
            current_vote = -1

        ## update the model state
        self.model_state.votes_round.append(current_vote)
        
    
    def act(self):
        
        ## what if no one was chosen, then no action
        if self.model_state.check_who_was_chosen() == -1:
            ## then the chooser tosses a coin
            return

        if not self.model_state.check_vote_pass():
            ## then the chooser tosses a coin
            return 

        ## action for chooser 
        if self.id == self.model_state.check_whose_turn():

            action = random.randint(0, 1)
            self.model_state.action_round[0] = action

        ## action for chosen
        elif self.id == self.model_state.check_who_was_chosen():

            action = random.randint(0, 1)
            self.model_state.action_round[1] = action

        else:
            ## If I am neither
            return

    def update(self):

        ## DO NOT CHANGE
        if self.id == 0:
            self.model_state.outcome_determination()
        ## DO NOT CHANGE

        current_outcome = self.model_state.check_outcome_current_round()

        if current_outcome == 1:
            return
        else: 
            chooser = self.model_state.check_whose_turn()
            self.belief[chooser] = 0

In [84]:
model = Model(N, coin_toss)

for i in range(rounds):
    model.step()

In [85]:
model.action_record

[[1, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 0],
 [1, 0],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1]]

In [86]:
model.check_outcome()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1]

In [15]:
# Example lists
items = ['apple', 'banana', 'cherry']
probabilities = [0.1, 0.6, 0.3]

# Using random.choices() for a single selection
selected_item = random.choices([0,1], weights=[coin_toss, 1-coin_toss], k=1)[0]
selected_item

0